In [19]:
import pandas as pd
def before_append_files():
    while True:
        while True:
            try:
                n = int(input('Enter the Number of Files We Have To Append And Create Master File: '))
                break
            except ValueError as e:
                print(f"Invalid input. Please enter a valid number. Error: {e}")

        while True:
            try:
                file_paths = []
                for i in range(n):
                    while True:
                        file_path = input(f'Enter the file path for file {i + 1}: \n')
                        if not file_path.endswith(".csv"):
                            print('Invalid File Path provided. Please enter a valid .csv file path.')
                        elif not file_path.strip():
                            print('File Path cannot be empty.')
                        else:
                            file_paths.append(file_path)
                            break

                master_df = pd.DataFrame()
                for path in file_paths:
                    df = pd.read_csv(path)
                    master_df = pd.concat([master_df, df], axis=0)

                break  # Break the while loop if no exception occurred
            except Exception as e:
                print(f"An error occurred: {e}, while opening below file path \n{file_path}")
                print("Please re-enter all the file paths.")

        while True:
            file_name = input('Enter the name for the file to save the master data: \n')
            if not file_name.strip():
                print('File name cannot be empty.')
            else:
                master_df.to_csv(f"{file_name}.csv", index=False)
                print(f"Master file saved as {file_name}.csv")
                break

        choice = input('Do you have more files to append and create a master file? (yes/no): ').strip().lower()
        if choice != 'yes':
            break



In [20]:
def mapping_data(df1, df2, left_key, right_key):
    #     static i=1
    concated_data = pd.merge(df1, df2, left_on=left_key, right_on=right_key, how='inner', suffixes=['_left', '_right'])
    print("\n:")
    #     print(f'{df2.columns}\n are all the columns Of Main File: ')

    l = len(df2.columns)
    df1_cols = list(concated_data.columns)
    df1_cols = df1_cols[:-l]

    while True:
        print(f'{list(concated_data.columns[-l:])}, are the columns of the mapping file: ')
        cols = input('Enter Columns You Want From Mapping File:\n')
        cols = [column.strip() for column in cols.split(',')]
        unvalid = []

        for col in cols:
            if col not in concated_data.columns[-l:]:
                unvalid.append(col)
        if len(unvalid) == 0:
            break
        else:
            print(f"{unvalid} columns are not present in mapping file, please provide valid keys: ")

    selected_cols = df1_cols + cols

    concated_data = concated_data[selected_cols]
    while True:
        print(f"{list(concated_data.columns)}:\nAre there the columns you want to rename? (yes/no): ")
        choice = input().strip().lower()
        if choice != 'yes':
            break

        cols_to_rename = {}
        while True:
            col_name = input('Enter the column name you want to rename: ').strip()
            if not col_name:
                print('Column Name Cannot Be Blank')
                print(f'-----------------------Choose From {concated_data.columns}-----------------------')
            
            elif col_name not in concated_data.columns:
                print(f"Provided Column Name Not Found In The Data. Please Choose From Above Provided Columns:\n")
            
            else:
                new_name = input('Enter the new column name: ').strip()
                if not new_name:
                    print('New Name Cannot Be Blank')
                
                else:
                    cols_to_rename[col_name] = new_name
                    _more = input('Enter yes if you want to rename more columns (yes/no): ').strip().lower()
                    if _more != 'yes':
                        break

        concated_data.rename(columns=cols_to_rename, inplace=True)
        print("Columns have been renamed successfully.")
        another_round = input('Do you want to rename more columns? (yes/no): ').strip().lower()
        if another_round != 'yes':
            print(f'{list(concated_data.columns)} are the updated column names')
            break

    return concated_data



# ---------------------------------------------------------------------------------------------------------------------------------------------------


In [21]:
def enter_file_paths():
    print('Enter the Path of the main file: ')
    main_file_path = input().strip()
    print('Enter the Mapping file Path: ')
    mapping_file_path = input().strip()
    return main_file_path, mapping_file_path



In [22]:
# ---------------------------------------------------------------------------------------------------------------------------------------------------


def enter_keys(df1, df2):
    while True:
        print('Enter the Key from the main file which we will use to map data from the mapping file: ')
        left_key = input().strip()
        
        if left_key not in df1.columns:
            print(f'Entered Key Not Present In File, Choose From Below Columns: {df1.columns}')
        else:
            break
    while True:
            print('Enter the key from the mapping file which will be used to map the data from mapping file:')
            right_key=input().strip()
            if right_key not in df2.columns:
                print(f'Entered Key Not Present In File, Choose From Below Columns: {df2.columns}')
            else:
                return left_key,right_key

# ---------------------------------------------------------------------------------------------------------------------------------------------------



In [23]:


def read_csv_files(main_file_path, mapping_file_path):
    try:
        df1 = pd.read_csv(main_file_path)
    except Exception as e:
        print(f'{e} Error Raised While Opening Main File')
        return False, None, None

    try:
        df2 = pd.read_csv(mapping_file_path)
        return True, df1, df2

    except Exception as e:
        print(f'{e} Error Raised While Opening Mapping File')
        return False, df1, None


# ---------------------------------------------------------------------------------------------------------------------------------------------------


In [24]:



def append_files(n: int):
        collect_df = []
        for i in range(n):
            while True:
                file_path = input(f'Enter the Absolute Path Of Master File {i + 1}: ')
                if not file_path.strip():
                    print("File Path Cannot Be Empty")
                elif not file_path.endswith('.csv'):
                    print('Invalid File Path Provided. ')

                else:
                    try:
                        df = pd.read_csv(file_path)
                        collect_df.append(df)
                        break
                    except Exception as e:
                        print(f'{e} Error Raised While Opening CSV File')

        if collect_df:
            master_df = pd.DataFrame(columns=collect_df[0].columns)
            for df in collect_df:
                master_df = pd.concat([master_df, df], axis=0)
            print("All the Data Is Appended In Master DF: ")
            return master_df
        else:
            print('No Valid Files were Provided. ')


# ---------------------------------------------------------------------------------------------------------------------------------------------------


In [25]:
#### Beginning the analysis of the columns
def begin_analysis(mapping_dict,cols_we_want,df,total_transactions):
    result={}
    for key in mapping_dict:
        # analysis of the description columns-------------->
        if key=='Description':
            desc_cols=mapping_dict[key]
            for column in  desc_cols:
                nas=pd.to_numeric(df[column],errors='coerce').notnull().sum()
                percentage=((total_transactions-nas)/total_transactions)*100
                result[column]={'Percentage Population':percentage,'NA Count':nas,'Comment':None,
                               'Column Type':'Important'}

        #: analysis of non description type columns--------------->
        else:
            value=mapping_dict[key]
            if value is None:
                result[key]={'Percentage Population':None,'NA Count':None,'Comment':None,
                               'Column Type':'Important'}
            else:
                value=mapping_dict[key]
                nas=df[value].isna().sum()
                percentage=((total_transactions-nas)/total_transactions)*100
                result[key]={'Percentage Population':percentage,'NA Count':nas,'Comment':None,
                               'Column Type':'Important'}


#: Analysis of all the Good To have columns
    for column in df.columns:
        if column not in cols_we_want:
            nas=df[column].isna().sum()
            percentage=((total_transactions-nas)/total_transactions)*100
            result[column]={'Percentage Population':percentage,'NA Count':nas,'Comment':None,
                               'Column Type':'Good To Have'}

        
        

    return result

In [26]:
#### Here We Are Now taking input of our mapping column from Users
def creating_mapping(df,mapped_columns):
    columns_we_want=[]
    mapping_dict={}
    print(f'We Will Be Creating Mapping For {mapped_columns.keys()}, Enter None If We Dont Have A Mapping Column For A Specific Column')
    for column in mapped_columns:
        #   ----------------------------if column name is description type--------------------------------
        if column=='Description':
            while True:
                while True:
                    invalid=[]
                    value=input(f'Enter the mapping column for {column}:  ')
                    values=[col.strip() for col in value.split(',')]
                    for value in values:
                        if value not in df.columns:
                            invalid.append(value)
                    if len(invalid)!=0:
                        print(f'{invalid} not present in master file')
                        print(f'Choose From {df.columns}')
                    else:
                        break
                mapping_dict[column]=values
                columns_we_want.append(values)
                break

    #   --------------------------------if column name is not description type--------------------------------      
        else:
            while True:
                value=input(f'Enter the mapping column for {column},(None if no mapping column): ')
                if not value.strip():
                    print('Column Name Cannot Be Empty')
                    print(f'Choose from {df.columns}')
                else:
                    if value.lower().strip()=='none':
                        mapping_dict[column]=None
                        break
                    
                    elif value not in df.columns:
                        print(f'{value} not present in Master file')
                        print(f'Choose From {df.columns}')

                    else:
                        mapping_dict[column]=value
                        columns_we_want.append(value)
                        break

                    
    cols_we_want=[]
    for column in columns_we_want:
        if isinstance(column,list):
            for col in column:
                cols_we_want.append(col)
        else:
            cols_we_want.append(column)
        
        
    return mapping_dict,cols_we_want

In [27]:
from typing import List
import numpy as np
import pandas as pd


def main():
#     before_append_files()
    print('---------------------Successfully Appended Multiple Files In Single Master File:-------------------')
    while True:

        while True:
            main_file_path, mapping_file_path = enter_file_paths()
            success, df1, df2 = read_csv_files(main_file_path, mapping_file_path)

            if success:
                break


        print('--------------------------Provided Path Files Opended Successfully:-------------------------- ')
        left_key, right_key = enter_keys(df1, df2)

        print('---------------------------Keys Received for mapping Data-----------------------------------------')
        concated_df = mapping_data(df1, df2, left_key, right_key)
        while True:
            print(
                'Are there more details you want to map from the same file with the same key in same mapping file? (yes/no)')
            if input().strip().lower() != 'yes':
                break
            else:
                left_key, right_key = enter_keys(concated_df, df2)
                concated_df = mapping_data(concated_df, df2, left_key, right_key)

                choice = input('Yes/No If there are more details to map?: ')
                if choice.strip().lower() != 'yes':
                    break
        print(
            '--------------------------------Successfully Mapped Details From Same Key Category---------------------------')

        while True:
            print('Are there more details you want to map from other mapping file in this file? (yes/no): ')
            if input().strip().lower() != 'yes':
                break
            else:
                while True:

                        mapping_file_path = input('Enter the New Mapping File Path: ')
                        if not mapping_file_path.strip():
                            print('File path cannot be empty..')
                        else:
                            try:
                                if mapping_file_path.strip().endswith('.csv'):
                                   new_mapping_file = pd.read_csv(mapping_file_path)
                                   break
                                else:
                                    new_mapping_file = pd.read_excel(mapping_file_path)
                                    break
                            except Exception as e:
                                print(f'{e} Error Raised While opening file..')

                left_key, right_key = enter_keys(concated_df, new_mapping_file)
                concated_df = mapping_data(concated_df, new_mapping_file, left_key, right_key)

                choice = input('Yes/No If there are more details to map?: ')
                if choice.strip().lower() != 'yes':
                    break


        file_name = input('Enter The File Name To Save the processed File: ')
        concated_df.to_csv(f'{file_name}.csv', index=False)

        print('Enter Yes If You Want To Map Details In Some New Master File Now (yes/no)?: ')
        if input().strip().lower() != 'yes':
            break

    # ----------------------------------------------------------------------------------------------------------------------------------------
    print('------------------Mapping Task Ended Here, Begining Analyzing the Files----------------------------')
    analyzing_cols = ['Date',
                  "Spend",
                  "Currency",
                  "Invoice Number",
                  "Invoice Line Number",
                  "Supplier Name",
                 'Description']
    mapped_columns={}
    for col in analyzing_cols:
        mapped_columns[col]=None




    # ---------------------------------taking the input of the path-----------------------------------------
    print('taking the input of the files:--->')
    while True:
        try:
            no_files = int(input('Enter the Number of Master File We Have: '))

            if isinstance(no_files,int):
                    break

        except Exception as e:
            print(f'{e} error raised in the input, Integer Value is Expected')



    if no_files == 1:
        while True:
            file_path = input('Enter the file path for master file: ')
            if not file_path.strip():
                print('File Path Cannot Be Empty')
            else:
                
                try:
                    if file_path.strip().endswith('.csv'):
                        df = pd.read_csv(file_path)
                        break
                        
                    else:
                        df = pd.read_excel(file_path)
                        break
                        
                except Exception as e:
                    print(f'{e} Error While Opening File File: ')
    else:
        df=append_files(no_files)



    print('--------------------------Initiating the Mapping Process----------------------------------------')               

    mapping_dict,columns_we_want=creating_mapping(df,mapped_columns)


    print('-------------------------Initiating the analysis-----------------------------------------------')
    result=begin_analysis(mapping_dict,columns_we_want,df,df.shape[0])
    df2=pd.DataFrame.from_records(result)
    df2=df2.transpose()
    df2.reset_index(inplace=True)
    print(df2.head(20))

    print('-----------------------Analysis End Below Are The Results-----------------------------------')
    while True:
        file_name=input('Enter a file name to save the results in: ->')
        if not file_name.strip():
            print('File Name Cannot Be Empty')
        else:
            df2.to_csv(f"{file_name}.csv",index=False)
            break

In [28]:
if __name__ == "__main__":
    main()

---------------------Successfully Appended Multiple Files In Single Master File:-------------------
Enter the Path of the main file: 
C:\Users\AnkitS-Simfoni\Users\ankit-Simfoni\Automation Tasks\Task 1 - By Ashish - Finalized\Mappings Task Files\Supplier Master .csv
Enter the Mapping file Path: 
C:\Users\AnkitS-Simfoni\Users\ankit-Simfoni\Automation Tasks\Task 1 - By Ashish - Finalized\Mappings Task Files\po master.csv
--------------------------Provided Path Files Opended Successfully:-------------------------- 
Enter the Key from the main file which we will use to map data from the mapping file: 
Material Code
Enter the key from the mapping file which will be used to map the data from mapping file:
Material Code
---------------------------Keys Received for mapping Data-----------------------------------------

:
['Material Code', 'Document ID', 'Document Line Description', 'Document Header Description', 'Document Date', 'Document Currency'], are the columns of the mapping file: 
Enter

Enter a file name to save the results in: ->DEMO_DATA_ANALYSIS_RESULT
